In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
import jenkspy
import json
from jenkspy import jenks_breaks

In [2]:
#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\ADMIN\OneDrive - Universidad de Guadalajara\3. Voces\2024\Tipologías\Nacional\nacional_subtipologias.gpkg"

# Ruta del archivo shp de polígonos
polygons_path = r"C:\Users\ADMIN\Desktop\Ejercicio\PCU_2018_SHP\PCUS_2018.shp"

In [3]:
# Leer el archivo de polígonos
polygons_gdf = gpd.read_file(polygons_path)

# Define el CRS proyectado que deseas utilizar
CRS_PROYECTADO = 'EPSG:6372' 

# Proyecta el GeoDataFrame a un CRS proyectado
polygons_gdf = polygons_gdf.to_crs(CRS_PROYECTADO)

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta)

# Proyecta MAPA_TIPO_gdf al mismo CRS que polygons_gdf si no lo está ya
if MAPA_TIPO_gdf.crs != CRS_PROYECTADO:
    MAPA_TIPO_gdf = MAPA_TIPO_gdf.to_crs(CRS_PROYECTADO)

In [4]:
print(MAPA_TIPO_gdf.columns)

Index(['fid', 'CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_AGEB', 'CVE_MZA',
       'AMBITO', 'TIPOMZA', 'NOMGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010',
       'VIVTOT2020', 'VIVPARDES2010', 'VIVPARDES2020', 'VIV1DO2010',
       'VIV1DO2020', 'VIV2YMDO2010', 'VIV2YMDO2020', 'VIV1C2010', 'VIV1C2020',
       'VIV2C2010', 'VIV2C2020', 'VIV3CYM2010', 'VIV3CYM2020', 'CVEMUN',
       'CVE_MET', 'NOM_MET', 'TIPO_MET', 'TIPO_MUN', 'area_ha', 'FILTRADO',
       'DESHABITACION', 'PROM_DESHAB_MUN', 'TASA_POP_MUN', 'DIFPOP', 'DIFVIV',
       'RELPOP', 'RELVIV', 'CVEPOP', 'CVEVIV', 'CVE', 'TIPOLOGIA',
       'DENS_POP20', 'DENS_POP10', 'DENS_VIV20', 'DENS_VIV10', 'CUARTOS2010',
       'CUARTOS2020', 'CPERC2010', 'CPERC2020', 'HABXVIV2010', 'HABXVIV2020',
       'R_DESHAB', 'SUBTIPOLOGÍA', 'geometry'],
      dtype='object')


In [ ]:
# Realizar una unión espacial manteniendo todas las geometrías de MAPA_TIPO_gdf
MAPA_TIPO_gdf_JOINED = gpd.sjoin(
    MAPA_TIPO_gdf, 
    polygons_gdf, 
    how='left', 
    predicate='intersects'
)

In [7]:
# Eliminar la columna 'fid' si existe para evitar conflictos de restricción única
if 'fid' in MAPA_TIPO_gdf_JOINED.columns:
    MAPA_TIPO_gdf_JOINED = MAPA_TIPO_gdf_JOINED.drop(columns='fid')

# Guardar la capa concatenada en el GeoPackage base
MAPA_TIPO_gdf_JOINED.to_file(MAPA_TIPO_ruta, layer='PCU_Tipologias', driver='GPKG', overwrite=True)

In [2]:
MAPA_TIPO_ruta = r"C:\Users\ADMIN\OneDrive - Universidad de Guadalajara\3. Voces\2024\Tipologías\Nacional\nacional_subtipologias.gpkg"

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='PCU_Tipologias')

In [7]:
print(MAPA_TIPO_gdf)

                  CVEGEO CVE_ENT CVE_MUN CVE_LOC CVE_AGEB CVE_MZA  AMBITO  \
0       0100100010229006      01     001    0001     0229     006  Urbana   
1       0100100010229007      01     001    0001     0229     007  Urbana   
2       0100100010229008      01     001    0001     0229     008  Urbana   
3       0100100010229010      01     001    0001     0229     010  Urbana   
4       0100100010233006      01     001    0001     0233     006  Urbana   
...                  ...     ...     ...     ...      ...     ...     ...   
955045         320500014      32     050    0014        0       0   Rural   
955046         320500003      32     050    0003        0       0   Rural   
955047         320500008      32     050    0008        0       0   Rural   
955048         320500009      32     050    0009        0       0   Rural   
955049         320500006      32     050    0006        0       0   Rural   

       TIPOMZA                    NOMGEO  POPTOT2010  ...  R_DESHAB  \
0   

In [18]:
# Filtrar la información donde 'CVE_AGEB' es igual a '0'
filtered_gdf = MAPA_TIPO_gdf[MAPA_TIPO_gdf['CVE_AGEB'] == '0'].copy()  # Copiamos el DataFrame para evitar la advertencia

# Convertir los valores NULL en 'VERSION1' a una cadena representativa
filtered_gdf['VERSION1'] = filtered_gdf['VERSION1'].fillna('NULL')

# Agrupar por 'NOM_MET' y 'VERSION1' (incluyendo los valores 'NULL' convertidos)
grouped_df = filtered_gdf.groupby(['NOM_MET', 'VERSION1']).agg({
    'area_ha': 'sum',
    'POPTOT2010': 'sum',
    'POPTOT2020': 'sum',
    'VIVTOT2010': 'sum',
    'VIVTOT2020': 'sum',
    'geometry': 'count'  # Contar la cantidad de polígonos
}).rename(columns={'geometry': 'count_polygons'})

# Resetear el índice para que 'NOM_MET' y 'VERSION1' sean columnas normales
grouped_df = grouped_df.reset_index()

# Calcular los porcentajes
grouped_df['pct_area_ha'] = (grouped_df['area_ha'] / grouped_df.groupby('NOM_MET')['area_ha'].transform('sum')) * 100
grouped_df['pct_POPTOT2010'] = (grouped_df['POPTOT2010'] / grouped_df.groupby('NOM_MET')['POPTOT2010'].transform('sum')) * 100
grouped_df['pct_POPTOT2020'] = (grouped_df['POPTOT2020'] / grouped_df.groupby('NOM_MET')['POPTOT2020'].transform('sum')) * 100
grouped_df['pct_VIVTOT2010'] = (grouped_df['VIVTOT2010'] / grouped_df.groupby('NOM_MET')['VIVTOT2010'].transform('sum')) * 100
grouped_df['pct_VIVTOT2020'] = (grouped_df['VIVTOT2020'] / grouped_df.groupby('NOM_MET')['VIVTOT2020'].transform('sum')) * 100
grouped_df['pct_count_polygons'] = (grouped_df['count_polygons'] / grouped_df.groupby('NOM_MET')['count_polygons'].transform('sum')) * 100


# Verificar el DataFrame resultante
print(grouped_df.head())

# Exportar el resultado a un archivo Excel
output_path = r"C:\Users\ADMIN\Desktop\Ejercicio\localidades.xlsx"
grouped_df.to_excel(output_path, index=False)

          NOM_MET VERSION1      area_ha  POPTOT2010  POPTOT2020  VIVTOT2010  \
0        Acapulco   2018.0   359.758210       14798       15306        4956   
1        Acapulco     NULL  1438.239409       61608       64325       17082   
2        Acayucan   2018.0    64.161176         637        1726         244   
3        Acayucan     NULL   928.870265       26701       26764        8012   
4  Aguascalientes   2018.0  1124.937888       30055       38277        7600   

   VIVTOT2020  count_polygons  pct_area_ha  pct_POPTOT2010  pct_POPTOT2020  \
0        5999              26    20.008826       19.367589       19.221158   
1       22122              81    79.991174       80.632411       80.778842   
2         581               7     6.461142        2.330090        6.058266   
3        9906              54    93.538858       97.669910       93.941734   
4       12286              65    43.069390       45.110694       46.547007   

   pct_VIVTOT2010  pct_VIVTOT2020  pct_count_polygons  


In [2]:
# Rutas de los archivos
archivo_1 = r"C:\Users\ADMIN\OneDrive - Universidad de Guadalajara\3. Voces\2024\Tipologías\Nacional\Estadísticas\resultados_metropoli_v2.xlsx"
archivo_2 = r"C:\Users\ADMIN\Desktop\Ejercicio\localidades_PCU.xlsx"

# Leer los archivos Excel
df1 = pd.read_excel(archivo_1)
df2 = pd.read_excel(archivo_2)

# Verificar las primeras filas de los DataFrames leídos
print("Contenido del primer archivo (df1):")
print(df1.head())
print("\nContenido del segundo archivo (df2):")
print(df2.head())

# Unir los DataFrames en base al campo 'NOM_MET'
merged_df = pd.merge(df2, df1[['NOM_MET', 'area_ha', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020']], on='NOM_MET', how='left')

# Verificar las primeras filas del DataFrame resultante
print("\nContenido del DataFrame unido (merged_df):")
print(merged_df.head())

# Exportar el DataFrame resultante a un nuevo archivo Excel
output_path = r"C:\Users\ADMIN\Desktop\Ejercicio\localidades_merged.xlsx"
merged_df.to_excel(output_path, index=False)

Contenido del primer archivo (df1):
            NOM_MET            TIPO_MET  POPTOT2010  POPTOT2020  VIVTOT2010  \
0  Ciudad de México  Zona Metropolitana    19040698    21167227     5841728   
1         Monterrey  Zona Metropolitana     4109950     5252175     1303759   
2       Guadalajara  Zona Metropolitana     4239068     5054743     1273345   
3   Puebla-Tlaxcala  Zona Metropolitana     2219608     2682810      692329   
4            Toluca  Zona Metropolitana     1314584     2139131      368634   

   VIVTOT2020  VIVPARDES2010  VIVPARDES2020  VIV1DO2010  VIV1DO2020  ...  \
0     6919090         151209         614096     1634291     1844845  ...   
1     1790339          45215         222113      249881      339234  ...   
2     1706284          43394         223822      224886      302462  ...   
3      883484          24253         106807      167820      188711  ...   
4      666316          13872          88222       84610      130936  ...   

   CUARTOS2010  CUARTOS2020  CPE

In [2]:
# Ruta del archivo
ruta_localidades = r"C:\Users\ADMIN\Desktop\Ejercicio\localidades_PCU.xlsx"

# Cargar el archivo Excel en un DataFrame
df_localidades = pd.read_excel(ruta_localidades)

# Filtrar las filas donde VERSION1 es TOTAL para obtener los totales
totales_df = df_localidades[df_localidades['PCU'] == 'Total'].copy()

# Renombrar las columnas para los totales
totales_df = totales_df.rename(columns={
    'area_ha': 'total_area_ha',
    'POPTOT2010': 'total_POPTOT2010',
    'POPTOT2020': 'total_POPTOT2020',
    'VIVTOT2010': 'total_VIVTOT2010',
    'VIVTOT2020': 'total_VIVTOT2020'
})

# Seleccionar solo las columnas necesarias más la clave de unión
totales_df = totales_df[['NOM_MET', 'total_area_ha', 'total_POPTOT2010', 'total_POPTOT2020', 'total_VIVTOT2010', 'total_VIVTOT2020']]

# Unir los totales con el DataFrame original basado en NOM_MET
df_localidades = pd.merge(df_localidades, totales_df, on='NOM_MET', how='left')

# Calcular los porcentajes
df_localidades['pct_area_ha'] = (df_localidades['area_ha'] / df_localidades['total_area_ha']) * 100
df_localidades['pct_POPTOT2010'] = (df_localidades['POPTOT2010'] / df_localidades['total_POPTOT2010']) * 100
df_localidades['pct_POPTOT2020'] = (df_localidades['POPTOT2020'] / df_localidades['total_POPTOT2020']) * 100
df_localidades['pct_VIVTOT2010'] = (df_localidades['VIVTOT2010'] / df_localidades['total_VIVTOT2010']) * 100
df_localidades['pct_VIVTOT2020'] = (df_localidades['VIVTOT2020'] / df_localidades['total_VIVTOT2020']) * 100

# Verificar el DataFrame resultante
print(df_localidades.head())

# Exportar el DataFrame resultante a un archivo Excel
output_path = r"C:\Users\ADMIN\Desktop\Ejercicio\localidades_calculado.xlsx"
df_localidades.to_excel(output_path, index=False)

          NOM_MET     PCU      area_ha  POPTOT2010  POPTOT2020  VIVTOT2010  \
0        Acapulco  DENTRO   359.758210       14798       15306        4956   
1        Acapulco   FUERA  1438.239409       61608       64325       17082   
2        Acayucan  DENTRO    64.161176         637        1726         244   
3        Acayucan   FUERA   928.870265       26701       26764        8012   
4  Aguascalientes  DENTRO  1124.937888       30055       38277        7600   

   VIVTOT2020  count_polygons  total_area_ha  total_POPTOT2010  \
0        5999            26.0    9983.062357            789971   
1       22122            81.0    9983.062357            789971   
2         581             7.0    2268.122131            112996   
3        9906            54.0    2268.122131            112996   
4       12286            65.0   10048.784086            932369   

   total_POPTOT2020  total_VIVTOT2010  total_VIVTOT2020  pct_area_ha  \
0            779566            293572            331319     3.